In [1]:
from __future__ import print_function
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize

In [2]:
#opens json file and creates a list of tuples with name of video and description called documents
#data represents a list of dictionarys for each video with all of the info from ted_main

In [3]:
documents=[]
with open("ted_main.json", encoding="utf8") as f:
    data=json.load(f)
    for x in data:
        documents.append((x["name"], x["description"]))

In [4]:
#given the data file and the title of the video, outputs the index that video is located at

In [5]:
def findindex(data,title):
    for x in range(len(data)):
        if data[x]['name']== title:
            return x

In [6]:
#creates tf-idf matrix, can alter max_df and min_df

In [7]:
vectorizer = TfidfVectorizer(stop_words = 'english', max_df = .8,
                            min_df = 40)
my_matrix = vectorizer.fit_transform([x[1] for x in documents]).transpose()

In [8]:
#runs svd, can alter k but values mostly live in the space under 30

In [16]:
words_compressed, _, docs_compressed = svds(my_matrix, k=25)
docs_compressed = docs_compressed.transpose()
print(docs_compressed.shape)

(2550, 25)


In [10]:
#creates clusters of 15 vidoes based on the index of the input video

In [18]:
docs_compressed = normalize(docs_compressed, axis = 1)
def closest_projects(project_index_in, k = 15):
    print(docs_compressed.shape)
    print(docs_compressed[project_index_in,:].shape)
    sims = docs_compressed.dot(docs_compressed[project_index_in,:])
    asort = np.argsort(-sims)[:k+1]
    return [(documents[i][0],sims[i]/sims[asort[0]]) for i in asort[1:]]

In [12]:
for i in range(10):
   print(documents[i][0])
   for title, score in closest_projects(i):
       print("{}:{:.3f}".format(title[:40], score))
    print()

In [13]:
#function that takes in the closest_projects function and the index of the desirable video 
#outputs the url of the second video in the cluster

In [14]:
def extract_cluster_ratings(index):
    lst=closest_projects(index)
    name=lst[1][0]
    index=findindex(data,name)
    return data[index]["url"]